# Testing BertPipeline

In [1]:
import sys
sys.path.append('../ml/BERT')
import masking
import BERT
from Vectorisation import Vectorisation
from Config import Config

import pickle
import tensorflow as tf
import numpy as np
with open("../../data/ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config(EPOCH=50)
vec = Vectorisation(config=config)

In [2]:
# Load the sequences used in src\script_oversample.py
with open('../../data/sequences.pkl', 'rb') as f:
    decoded_sequences = pickle.load(f)

decoded_sequences[0][0:3]

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 7.889],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 75.141, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 3.25]]

In [3]:
seps = vec.sep_from_seq(decoded_sequences)
encoded_sequences = vec.encode(decoded_sequences, seps)
print(encoded_sequences.shape)
print(encoded_sequences[0])

(254, 128)
[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [4]:
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded_sequences, config.TOKEN_DICT)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.batch(config.BATCH_SIZE)
print(mlm_ds)

(254, 128) (254, 128) (254, 128)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.float64, name=None))>


In [5]:
bert_masked_model = BERT.create_masked_language_bert_model(config)
#bert_masked_model.summary()

bert_masked_model.fit(mlm_ds, epochs=config.bert.epoch)


Epoch 1/50
16/16 [==============================] - 2s 17ms/step - loss: 3.2383
Epoch 2/50
16/16 [==============================] - 0s 16ms/step - loss: 2.7221
Epoch 3/50
16/16 [==============================] - 0s 15ms/step - loss: 2.5815
Epoch 4/50
16/16 [==============================] - 0s 14ms/step - loss: 2.5305
Epoch 5/50
16/16 [==============================] - 0s 14ms/step - loss: 2.4930
Epoch 6/50
16/16 [==============================] - 0s 14ms/step - loss: 2.4548
Epoch 7/50
16/16 [==============================] - 0s 14ms/step - loss: 2.4310
Epoch 8/50
16/16 [==============================] - 0s 17ms/step - loss: 2.3913
Epoch 9/50
16/16 [==============================] - 0s 17ms/step - loss: 2.3363
Epoch 10/50
16/16 [==============================] - 0s 14ms/step - loss: 2.2485
Epoch 11/50
16/16 [==============================] - 0s 14ms/step - loss: 2.1179
Epoch 12/50
16/16 [==============================] - 0s 14ms/step - loss: 2.0189
Epoch 13/50
16/16 [=================

In [6]:
from BERTPipeline import BERTPipeline

bert = BERTPipeline(config, vec)
bert_masked_model = bert.train(decoded_sequences)

Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_2[0][0]']             
                                                                                                  
 tf.__operators__.add_3 (TF  (None, 128, 64)              0         ['word_embedding[0][0]']      
 OpLambda)                                                                                        
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operato

In [7]:
new_sequences = bert.predict(y_masked_encoded[0:1], bert_masked_model)

print(new_sequences == y_masked_encoded[0:1])
print("Accuracy brut: ", np.sum(new_sequences == y_masked_encoded[0:1]) / (100*len(y_masked_encoded[0:1])))
print("Accuracy without taking in acount padding: ", np.sum((new_sequences == y_masked_encoded[0:1]) * (y_masked_encoded[0:1] != 0)) / np.sum(y_masked_encoded[0:1] != 0))
print("Accuracy on masked tokens: ", np.sum((new_sequences == y_masked_encoded[0:1]) * (x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]'])) / np.sum(x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]']))

1/1 [==============================] - 0s 166ms/step
[[ True False  True  True  True  True  True  True  True  True False  True
   True  True  True  True  True  True False  True  True  True  True  True
   True  True  True  True False  True  True False  True False False False
   True  True  True  True  True  True  True False  True False  True  True
   True  True False  True False False  True  True  True  True  True  True
  False  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True False  True  True  True False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False]]
Accuracy brut:  0.8
Accuracy without taking in acount padding:  0.8421052631578947
Accuracy on masked tokens:  0.8
